<a href="https://colab.research.google.com/github/olgOk/QCircuit/blob/master/tutorials/QFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Fourier Transform

by Olga Okrut


Install frameworks, and import libraries

In [1]:
!pip install qcircuit tensornetwork jax jaxlib colorama

     |████████████████████████████████| 245kB 2.8MB/s 
  Created wheel for qcircuit: filename=qcircuit-1.0.1-cp36-none-any.whl size=5988 sha256=cf2fd63859feedd5e806c078069e60aef00a4497e11b84f1478b50da09701274
  Stored in directory: /root/.cache/pip/wheels/a3/dd/39/d3dece77c172e493c47e48a0b28382d116ff2daad674a7270b
Successfully built qcircuit
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
from qcircuit import QCircuit as qc
import numpy as np

The quantum Fourier transform (QFT) is the quantum implementation of the discrete Fourier transform over the amplitudes of a wavefunction.

Let's say we have a given quantum circuit consisting of *n* qubits. The state vector of the sysytem is a vector of $ N = 2^n$ size. Let ``` |j>```, where $ j = 0,1,2,..N-1 $ be the basis states of the given system. The QFT of the basis states of the system is defined as:
$ |j> -> \frac{1}{\sqrt(N)} \displaystyle\sum_{k=0}^{N-1} exp(\frac{2i\pi jk}{N})|k>$

Let me come up with an example. Suppose, we have three qubits circuit. As an input we send a number 5 (or *|101>* or *|5>*). I want to calculate the output after QFT. 

The size of the vector state is $N = 2^3 = 8$. Following the notation above:
$ |out> = \frac{1}{\sqrt(8)} ( exp(\frac{2i\pi *5*0}{8})|000> + exp(\frac{2i\pi *5*1}{8})|100> + exp(\frac{2i\pi *5*2}{8})|010> +  exp(\frac{2i\pi *5*3}{8})|110> + exp(\frac{2i\pi *5*4}{8})|001> + exp(\frac{2i\pi *5*5}{8})|101> + exp(\frac{2i\pi *5*6}{8})|011> + exp(\frac{2i\pi *5*7}{8})|111>)$

After evaluating the expression above, we will have the following states with the amplitudes (rounded to three decimals):
$ 0.354     |000> \\
(-0.25 -0.25i)|100> \\
(0.354i)|010> \\
(0.25 - 0.25i)|110> \\
(-0.354)|001> \\
(0.25+0.25)|101> \\
(-0.354i)|011> \\
(-0.25+0.25i)|111> $

But how can we implement the QFT using quantum gates and QCircuit simulator? 

It has been proven that the QFT could be implement using Hadamard gates and controlled phase shift gates.

The phase shift gate (a simple rotation gate. On the Bloch sphere, this gate corresponds to rotating the qubit state around the z axis by the given angle) is defined as:

$ R = \begin{pmatrix} 
  1 & 0  \\
  0 & e^{i\phi} \end{pmatrix} $

 where $\phi = \frac{2 i pi}{2^k}$

 The phase shift gate has been implemented in to the QCircuit class as:  
 ```R(angle, target_qubit)```.

Thus, if  target qubit is 0, and controll qubit is 1, the controlled phase shift gate is found as:
$ CR = \begin{pmatrix} 
  1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 \\
  0 & 0 & 1 & 0 \\
  0 & 0& 0 & e^{i\phi} \end{pmatrix} $

  The controlled phase shift gate has been implemented in to the QCircuit class as:  
 ```CR(angle, [controll_qubits], target_qubit)```.

The example disscussed above can be implemented using three qubits quantum circuit as below:

![picture](https://drive.google.com/uc?id=1J5fT63NQJ6UHl7s8kh-hKZZ4x7NF4EMq)

Before we continue to the general case, let's create this circuit, send `|5>` on the input and compare our calculations with the output of the QCircuit class.

In [5]:
qft = qc.QCircuit(3)
# send |5> on the input
qft.X(0)
qft.X(2)
# apply the Hadamard and Controlled Shift Phase Gates
qft.H(0)
qft.CR(np.pi/2, control=[1], target=0)
qft.CR(np.pi/4, control=[2], target=0)
qft.H(1)
qft.CR(np.pi/2, control=[2], target=1)
qft.H(2)
output_state = qft.get_state_vector()
print("output state = ", output_state)

output state =  [ 0.354+0.j    -0.25 -0.25j   0.   +0.354j  0.25 -0.25j  -0.354+0.j
  0.25 +0.25j  -0.   -0.354j -0.25 +0.25j ]


In [6]:
qft.visualize()

    |  ╔═╗   ╔═╗   ╔═╗   ╔═╗               
q 0 |──║X║───║H║───║R║───║R║───────────────
    |  ╚═╝   ╚═╝   ╚╦╝   ╚═╝               
    |              ╔╩╗   ╔═╗   ╔═╗         
q 1 |──────────────║r║───║H║───║R║─────────
    |              ╚═╝   ╚═╝   ╚╦╝         
    |  ╔═╗               ╔═╗   ╔╩╗   ╔═╗   
q 2 |──║X║───────────────║r║───║r║───║H║───
    |  ╚═╝               ╚═╝   ╚═╝   ╚═╝   


As you can see that is exactly the same as we have calculated.
Now, let implement a larger QFT.

![picture](https://drive.google.com/uc?id=1X8IGjNUpm817TtwGdEhUfRYlA6q486-Y)

I will implement the QFT on 4 qubits. The input state has been chosen arbitrary ```|1010>``` (number 10).



In [8]:
num_qubits = 4
large_qft = qc.QCircuit(num_qubits)
large_qft.X(1)
large_qft.X(3)

for qubit in range(num_qubits):
  large_qft.H(qubit)
  for control in range(qubit + 1, num_qubits):
    phi = 2 * np.pi / 2 ** (control - qubit + 1)
    large_qft.CR(phi, control=[control], target=qubit)
output_state = large_qft.get_state_vector()
print("output_state = ", output_state)

output_state =  [ 0.25 +0.j    -0.096+0.231j -0.177-0.177j  0.231-0.096j  0.   +0.25j
 -0.231-0.096j  0.177-0.177j  0.096+0.231j -0.25 +0.j     0.096-0.231j
  0.177+0.177j -0.231+0.096j -0.   -0.25j   0.231+0.096j -0.177+0.177j
 -0.096-0.231j]


In [9]:
large_qft.visualize()

    |  ╔═╗   ╔═╗   ╔═╗   ╔═╗                           
q 0 |──║H║───║R║───║R║───║R║───────────────────────────
    |  ╚═╝   ╚╦╝   ╚═╝   ╚═╝                           
    |  ╔═╗   ╔╩╗         ╔═╗   ╔═╗   ╔═╗               
q 1 |──║X║───║r║─────────║H║───║R║───║R║───────────────
    |  ╚═╝   ╚═╝         ╚═╝   ╚╦╝   ╚═╝               
    |              ╔═╗         ╔╩╗   ╔═╗   ╔═╗         
q 2 |──────────────║r║─────────║r║───║H║───║R║─────────
    |              ╚═╝         ╚═╝   ╚═╝   ╚╦╝         
    |  ╔═╗               ╔═╗         ╔═╗   ╔╩╗   ╔═╗   
q 3 |──║X║───────────────║r║─────────║r║───║r║───║H║───
    |  ╚═╝               ╚═╝         ╚═╝   ╚═╝   ╚═╝   
